In [8]:
# update rmse 並記得重新啟動 Anaconda
# !pip install -U scikit-learn 

In [9]:
# Modules
import random
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras import optimizers, metrics
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers.advanced_activations import LeakyReLU
import math
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import r2_score
from keras import backend as K

import warnings
warnings.filterwarnings('ignore')

In [10]:

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2

In [11]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

In [12]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [13]:
def nn_ave(x,y):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train,X_val,Y_train,Y_val  = train_test_split(x, y, test_size=0.2, random_state=i)
        
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        batch_size = 32
        epochs = 1000
        model = build_model()
        history = model.fit(
            X_train,
            Y_train,
            epochs = epochs,
            validation_data = (X_val, Y_val),
            batch_size = batch_size,
            shuffle = True,
            verbose=0
            )
        
        y_pred = model.predict(X_val)
        rmse.append(math.sqrt(mean_squared_error(Y_val , y_pred )))
        r2.append(r2_score(Y_val , y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)

In [14]:
## 
n='Input_A3_018' # 改名稱
##
dire= ['./Data_Science_2020_imdb/BayesianRidge_Pre/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_1/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_2/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_3/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_5/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_6/','./Data_Science_2020_imdb/Data/BayesianRidge_Pre_7/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))

Input_A3_018
./Data_Science_2020_imdb/BayesianRidge_Pre/
feature number: 2
train_rmse: 0.009002384353568876
test_rmse: 0.009200611148695756
test_r2: 0.5040608356862285
feature number: 3
train_rmse: 0.00886286481825366
test_rmse: 0.008990604372643365
test_r2: 0.5259081520349198
feature number: 4
train_rmse: 0.00872477114064293
test_rmse: 0.008856737530801328
test_r2: 0.5399154298217667
feature number: 5
train_rmse: 0.00869561892758856
test_rmse: 0.008914010798877044
test_r2: 0.5339396878187016
feature number: 6
train_rmse: 0.00868114120896614
test_rmse: 0.008967017046202131
test_r2: 0.5282097318398581
feature number: 7
train_rmse: 0.00866609311480066
test_rmse: 0.008983455006245356
test_r2: 0.5265244001313747
feature number: 8
train_rmse: 0.008637234186114732
test_rmse: 0.009064466622629162
test_r2: 0.5181095158025865
feature number: 9
train_rmse: 0.008586194100706847
test_rmse: 0.009085810372583917
test_r2: 0.5160774476360562
feature number: 10
train_rmse: 0.008553826835961165
test_rms

train_rmse: 0.008526237193484682
test_rmse: 0.009221676254163607
test_r2: 0.5015790377472705
feature number: 15
train_rmse: 0.008467814995669858
test_rmse: 0.00927773009378962
test_r2: 0.49489464833552493
feature number: 16
train_rmse: 0.008458759785764176
test_rmse: 0.00928047376118266
test_r2: 0.4945021784264821
feature number: 17
train_rmse: 0.008444728781459378
test_rmse: 0.009302479098484286
test_r2: 0.49201602792396715
feature number: 18
train_rmse: 0.008404262759005802
test_rmse: 0.009311787473631695
test_r2: 0.49107277444778363
feature number: 19
train_rmse: 0.008389178324084201
test_rmse: 0.009353469542782003
test_r2: 0.4864952000232544
feature number: 20
train_rmse: 0.008347952286862359
test_rmse: 0.009433006727111306
test_r2: 0.4780496591655277
Test_rmse_min = 1.fea_num: 4  2.rmse: 0.008856737530801328
Test_r2_max = 1. fea_num: 4  2.r2: 0.5399154298217667
./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/
feature number: 2
train_rmse: 0.009002384353568876
test_rmse: 0.009200

train_rmse: 0.00868114120896614
test_rmse: 0.008967017046202131
test_r2: 0.5282097318398581
feature number: 7
train_rmse: 0.00866609311480066
test_rmse: 0.008983455006245356
test_r2: 0.5265244001313747
feature number: 8
train_rmse: 0.008637234186114732
test_rmse: 0.009064466622629162
test_r2: 0.5181095158025865
feature number: 9
train_rmse: 0.008586194100706847
test_rmse: 0.009085810372583917
test_r2: 0.5160774476360562
feature number: 10
train_rmse: 0.008553826835961165
test_rmse: 0.009145166464286864
test_r2: 0.5098474805206125
feature number: 11
train_rmse: 0.008547933866457235
test_rmse: 0.009167436639262436
test_r2: 0.5073923588510478
feature number: 12
train_rmse: 0.008546423825595673
test_rmse: 0.009182500583824804
test_r2: 0.5058032589243476
feature number: 13
train_rmse: 0.008530680116825506
test_rmse: 0.009189333785222645
test_r2: 0.5051375886043254
feature number: 14
train_rmse: 0.008526237193484682
test_rmse: 0.009221676254163607
test_r2: 0.5015790377472705
feature number: 

# 可以修改第一層的神經元數量

In [15]:
#
def build_model():
    model = models.Sequential()
#
    model.add(layers.Dense(7, activation='sigmoid',input_shape=(x.shape[1],))) # 調這裡layers的數量
#
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss=metrics.mean_squared_error, metrics=[r2_keras])

    return model

In [16]:
fea_num=[3,4,5,6,7] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/BayesianRidge_Pre/Input_A3_018.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 3
test_rmse_ave: 0.00921944356322162
[0.009185806639545456, 0.008568616625085141, 0.009139646449926956, 0.009569393662595807, 0.009684550759816377, 0.009602057652564248, 0.008896389866605486, 0.009581176929014348, 0.008262190401155405, 0.00970460664590695]


test_r2_ave: 0.5000437591266242
[0.5225473892671526, 0.600960856482819, 0.5138126328234441, 0.467407210081873, 0.46624828830976717, 0.4029354289338396, 0.5116473193263493, 0.48350885446240766, 0.5791195218065777, 0.4522500897720122]


feature number: 4
test_rmse_ave: 0.009174696759659448
[0.009045975189329827, 0.008509548737761306, 0.009139320107924644, 0.009718830101019442, 0.009814761591031618, 0.009717320729045756, 0.008762137357781329, 0.009481504738171725, 0.008161010083974716, 0.009396558960554116]


test_r2_ave: 0.504453202855649
[0.5369728486014418, 0.6064434585747545, 0.5138473520058852, 0.4506433053101092, 0.4517989909541875, 0.38851507105333427, 0.5262752486057258, 0.4941989887402851, 0.5893647636655026, 

In [17]:
fea_num=[3,4,5,6,7]   # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_1/Input_A3_018.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 3
test_rmse_ave: 0.009432264019081805
[0.009288280039540615, 0.008833998074629978, 0.00919307846986612, 0.009426321945619025, 0.011036443308569921, 0.009499495545617043, 0.009427484028455401, 0.009886571246504916, 0.008482096406374456, 0.009248871125640568]


test_r2_ave: 0.47620379529420676
[0.5118354081905951, 0.5758605431974915, 0.5081113380970841, 0.4832137163983308, 0.3068318573027665, 0.41562211810048433, 0.4515998396016231, 0.45005841466654617, 0.5564171031762343, 0.5024876142109116]


feature number: 4
test_rmse_ave: 0.009456892898654585
[0.009560906947568847, 0.00909459887382181, 0.009315687587826624, 0.010143207152158648, 0.010248657231246324, 0.009886185761174664, 0.009230113678645572, 0.00973598452911952, 0.008020095757983518, 0.009333491467000325]


test_r2_ave: 0.4736854347555308
[0.48275791152413294, 0.5504674198343522, 0.4949030932163665, 0.40162005254121835, 0.40225733184021906, 0.3670779642898542, 0.47432168486631865, 0.4666836345890767, 0.603423028826

In [18]:
fea_num=[3,4,5,6,7]  # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_2/Input_A3_018.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 3
test_rmse_ave: 0.009139847351882873
[0.00925360920199799, 0.008063635021585096, 0.008770417901239579, 0.00953866438809645, 0.009372489229165476, 0.009879381669425945, 0.009312365832811715, 0.00957515945872522, 0.008560161833843892, 0.009072588981937368]


test_r2_ave: 0.5071801178186089
[0.5154730002179935, 0.6466087435231624, 0.5523016921722389, 0.4708222461911701, 0.5000918452495009, 0.3679488719909255, 0.4649110038867148, 0.48415741656779543, 0.5482144515288447, 0.5212719068577443]


feature number: 4
test_rmse_ave: 0.0093524948996609
[0.009006295968479523, 0.010460538792844436, 0.008775252500824084, 0.009683438829711078, 0.00979204636414811, 0.009186921602474475, 0.009030943267415829, 0.010139997260237777, 0.008340750658526103, 0.009108763751947588]


test_r2_ave: 0.4867336312654082
[0.5410259797610171, 0.40529418628531, 0.551807978271666, 0.4546370024250569, 0.4543335609841044, 0.45344646989745907, 0.4967634334877038, 0.42150336673652256, 0.5710776655159069, 0.51

In [19]:
fea_num=[3,4,5,6,7] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_3/Input_A3_018.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 3
test_rmse_ave: 0.009380173548361863
[0.009683490304591973, 0.00861387251966344, 0.009134503889445906, 0.010036031384014023, 0.00950449209873612, 0.00964370204345944, 0.008908198200803505, 0.009861775576763957, 0.008389623816570898, 0.010026045649569366]


test_r2_ave: 0.4822922145529046
[0.46940944128224116, 0.5967346052780287, 0.5143596002697975, 0.4141985225938267, 0.4859111919297675, 0.39774522698653947, 0.5103500608154132, 0.4528134790315921, 0.566036343385659, 0.41536367395617957]


feature number: 4
test_rmse_ave: 0.009278015938085828
[0.008909609398394127, 0.008367334450568467, 0.00949924599095408, 0.00969549554048811, 0.01012709658472406, 0.00975394867549067, 0.009235731999903478, 0.00970769986584039, 0.008265780829917725, 0.00921821604457718]


test_r2_ave: 0.4926638508789908
[0.5508276612934061, 0.6194880219973555, 0.4748018999969623, 0.4532781096924632, 0.41635304349729174, 0.3838965862460423, 0.47368153489112264, 0.4697778798647888, 0.5787536455333742, 0.5

In [ ]:
fea_num=[3,4,5,6,7]  # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_4/Input_A3_018.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 3
test_rmse_ave: 0.009245811046592861
[0.00918320697290779, 0.008339527141238812, 0.009424304734042528, 0.00919323143845487, 0.009490973286636395, 0.009731743981809919, 0.008938934572046758, 0.0096445863044068, 0.00853982045199967, 0.009971781582385077]


test_r2_ave: 0.49660623103050644
[0.5228175978691756, 0.6220129437160236, 0.48305597629904773, 0.508455514437067, 0.48737259088032403, 0.38669849067492257, 0.5069653075946768, 0.4766498319962388, 0.5503590426175375, 0.42167501422005005]


feature number: 4
test_rmse_ave: 0.009519097178135523
[0.008982309363499514, 0.00948455699303618, 0.00961846826338997, 0.0100098261805687, 0.010241919861887926, 0.009521695394083837, 0.008966036796935743, 0.010193103945001624, 0.00866931007135912, 0.009503744911592598]


test_r2_ave: 0.46799870332129234
[0.5434675091167456, 0.5110908373663068, 0.46153595356379895, 0.4172537153597481, 0.40304297415805757, 0.412887602544733, 0.5039710818645523, 0.4154279234128977, 0.5366198166320277, 0.

In [ ]:
fea_num=[3,4,5,6,7]  # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_5/Input_A3_018.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

In [ ]:
fea_num=[3,4,5,6,7] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_6/Input_A3_018.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

In [ ]:
fea_num=[3,4,5,6,7] # 要測試的feature數量

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('./Data_Science_2020_imdb/Data/BayesianRidge_Pre_7/Input_A3_018.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=abs(numcorr['Predict']).sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')